In [6]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from math import sin, cos, sqrt, atan2, radians
import re
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


In [11]:
house_address_all = []
house_details_all = []
house_address = []
house_details = []
postal_code_array = []
latitude_array = []
longitude_array = []
year_built_array = []
payment_monthly = []
lot_size = []
basement = []
garage = []
mlsa = []

base_url = 'https://www.remax.ca/ab/calgary-real-estate?page='
urls = [base_url + str(x) for x in range(1,2)]

for url in urls:
    
    #browser.visit(url)
    time.sleep(5)

    # Parse HTML with Beautiful Soup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #time.sleep(5)

    try:
        addresses = soup.find_all('div', class_='left-content flex-one')
        for address in addresses:
            house_address_all.append(address.text)
    except:
        house_address_all.append('None')
        
    #time.sleep(5)
        
    try:
        details = soup.find_all('div', class_='property-details')
        for detail in details:
            house_details_all.append(detail.text)
    except:
        house_details_all.append('None')
        
    #time.sleep(5)
        
    house_details_more=[]
    url_house = []
        
    try:
        for detail in soup.find_all('a'):
            house_details_more.append(detail.get('href'))
    except:
        house_details_more.append('None')
    
    #time.sleep(5)

    for i in house_details_more:
        if '/ab/calgary-real-estate/' in str(i):
            link = 'https://www.remax.ca' + str(i)
            url_house.append(link)
            
    #time.sleep(5)
    
    for i in url_house:
    
        time.sleep(5)

        page = requests.get(i)
        soup = BeautifulSoup(page.text, "html.parser")

        lat_all = soup.find_all('script')[4]

        string = ', '.join(lat_all)
        pc = "postalCode"
        position_pc = string.find(pc)
        pcd = string[(position_pc+14):(position_pc+21)]
        postal_code_array.append(pcd)

        lat = "latitude"
        position_lat = string.find(lat)
        latd = string[(position_lat+11):(position_lat+20)]
        latitude_array.append(latd)

        long = "longitude"
        position_long = string.find(long)
        longd = string[(position_long+12):(position_long+23)]
        longitude_array.append(longd)

        house_address = []

        #time.sleep(5)

        try:
            addresses = soup.find_all('section', class_='property-details')
            for address in addresses:
                house_address.append(address.text)
        except:
            house_address.append('None')

        #time.sleep(5)

        string = ', '.join(house_address)
        s_year = "Year Built"
        if s_year in string:
            position_year = string.find(s_year)
            year_built = string[(position_year+12):(position_year+16)]
            year_built_array.append(year_built)
        else:
            year_built_array.append(np.nan)
        
        s_lot = "Lot Size"
        if s_lot in string:
            position_lot = string.find(s_lot)
            lot_size_temp = string[(position_lot+10):(position_lot+14)]
            lot_size.append(lot_size_temp)
        else:
            lot_size.append(np.nan)
        
        basement_yn = "Basement"
        if basement_yn in string:
            basement.append(1)
        else:
            basement.append(0)
            
        garage_yn = "Garage"
        if garage_yn in string:
            garage.append(1)
        else:
            garage.append(0)
        
        try:
            payment = soup.find(class_="payment-amount").text
            payment_monthly.append(payment)
        except:
            payment_monthly.append(np.nan)
        
        result = re.search('MLSÂ® #: (.*)Ownership', string)
        mlsa.append(result)

        #time.sleep(5)

In [10]:
print(mlsa)

['A1046628', 'A1046342', 'A1046598', 'A1046588', 'A1045693']

In [3]:
address_df = pd.DataFrame(house_address_all)

new_df = address_df[0].str.split(' ', 2, expand=True)
new_df["price"] = new_df[1].str.replace("$", "")
new_df["price"] = new_df["price"].str.replace(",", "")
new_df["price"] = pd.to_numeric(new_df["price"])

del new_df[0]
del new_df[1]
new_df.head()

,2,price
0,"86 DOVERCLIFFE CLOSE SE, Calgary, AB, T2B 1W3",339800
1,"53 SETON MANOR SE, Calgary, AB, T3M 2V8",448888
2,"3112 - 522 CRANFORD DR SE, Calgary, AB, T3M 2L7",254900
3,"712 - 235 9A ST NW, Calgary, AB, T2N 4H7",349900
4,"2532 HOLLY DR SE, Calgary, AB, T2B 1S8",388800


In [4]:
final_df_pc = new_df[2].str.split(', Calgary, AB, ', expand=True)
final_df_pc.head()

,0,1
0,86 DOVERCLIFFE CLOSE SE,T2B 1W3
1,53 SETON MANOR SE,T3M 2V8
2,3112 - 522 CRANFORD DR SE,T3M 2L7
3,712 - 235 9A ST NW,T2N 4H7
4,2532 HOLLY DR SE,T2B 1S8


In [5]:
final_df_pc_clean = final_df_pc[1].str.replace(" ", "")
#final_df_pc_clean = final_df_pc[1]
final_df_pc_clean.head()

0    T2B1W3
1    T3M2V8
2    T3M2L7
3    T2N4H7
4    T2B1S8
Name: 1, dtype: object

In [6]:
df_add = pd.concat([new_df, final_df_pc_clean, final_df_pc[0]], axis=1)
del df_add[2]
df_add.columns = ["price", "postal_code", "address"]
df_add.head()

,price,postal_code,address
0,339800,T2B1W3,86 DOVERCLIFFE CLOSE SE
1,448888,T3M2V8,53 SETON MANOR SE
2,254900,T3M2L7,3112 - 522 CRANFORD DR SE
3,349900,T2N4H7,712 - 235 9A ST NW
4,388800,T2B1S8,2532 HOLLY DR SE


In [7]:
details = pd.DataFrame(house_details_all)
details_df_temp = details[0].str.split('|', expand=True)
details_df_temp.head()

,0,1,2,3
0,4 bed,2 bath,1097 sqft,house
1,4 bed,3 + 1 bath,2077 sqft,house
2,2 bed,2 bath,831 sqft,condo
3,1 bed,1 bath,615 sqft,condo
4,4 bed,2 bath,1149 sqft,house


In [8]:
details_df_bed = details_df_temp[0].str.replace(' bed', '')
details_df_bath = details_df_temp[1].str.replace(' bath', '')
details_df_area = details_df_temp[2].str.replace(' sqft', '')

In [9]:
details_df_bath_all = details_df_bath.str.split('+', expand=True)
details_df_bath_full = details_df_bath_all[0]
details_df_bath_half = details_df_bath_all[1]

In [10]:
details_df_bed = details_df_bed.replace('N/A', np.nan)
details_df_bed = pd.to_numeric(details_df_bed)
details_df_area = details_df_area.replace('N/A', np.nan)
details_df_area = pd.to_numeric(details_df_area)
details_df_bath_full = details_df_bath_full.replace('N/A', np.nan)
details_df_bath_full = pd.to_numeric(details_df_bath_full)
details_df_bath_half = details_df_bath_half.replace('N/A', np.nan)
details_df_bath_half = pd.to_numeric(details_df_bath_half)

In [11]:
data = {'bed':details_df_bed, 'full_bath':details_df_bath_full, 'half_bath':details_df_bath_half,
       'property_area':details_df_area, 'property_type':details_df_temp[3]}

In [12]:
details_df = pd.DataFrame(data)
details_df.head()

,bed,full_bath,half_bath,property_area,property_type
0,4.0,2.0,NaN,1097.0,house
1,4.0,3.0,1.0,2077.0,house
2,2.0,2.0,NaN,831.0,condo
3,1.0,1.0,NaN,615.0,condo
4,4.0,2.0,NaN,1149.0,house


In [13]:
calgary_df_dup = pd.concat([df_add, details_df], axis=1)
calgary_df = calgary_df_dup.drop_duplicates()
calgary_df.head()

,price,postal_code,address,bed,full_bath,half_bath,property_area,property_type
0,339800,T2B1W3,86 DOVERCLIFFE CLOSE SE,4.0,2.0,NaN,1097.0,house
1,448888,T3M2V8,53 SETON MANOR SE,4.0,3.0,1.0,2077.0,house
2,254900,T3M2L7,3112 - 522 CRANFORD DR SE,2.0,2.0,NaN,831.0,condo
3,349900,T2N4H7,712 - 235 9A ST NW,1.0,1.0,NaN,615.0,condo
4,388800,T2B1S8,2532 HOLLY DR SE,4.0,2.0,NaN,1149.0,house


In [46]:
calgary_df.to_csv('data/calgary_listings-a.csv', index=False)

## Cleanup Postal Codes, Latitude, Longitude and Year Built

In [14]:
postal_codes_messy = pd.DataFrame(postal_code_array, columns=["postal_code"])

In [15]:
postal_codes_messy.head()

,postal_code
0,"T2B1W3"""
1,"T3M2V8"""
2,"T3M2L7"""
3,"T2N4H7"""
4,"T2B1S8"""


In [16]:
postal_code_array_clean = postal_codes_messy["postal_code"].str.replace('"', '')
postal_code_array_clean = postal_code_array_clean.str.replace(" ", "")

In [17]:
latitude_array_messy = pd.DataFrame(latitude_array, columns=["latitude"])

In [18]:
latitude_array_messy = latitude_array_messy["latitude"].str.replace(',', '')
latitude_array_messy.head()

0     51.02428
1     50.86897
2    50.869268
3    51.053719
4    51.030723
Name: latitude, dtype: object

In [19]:
longitude_array_messy = pd.DataFrame(longitude_array, columns=["longitude"])

In [20]:
longitude_array_messy = longitude_array_messy["longitude"].str.replace(',', '')
longitude_array_messy.head()

0    -113.979354
1    -113.957824
2    -113.970215
3    -114.084884
4    -113.986627
Name: longitude, dtype: object

<div class="alert alert-danger">
    <b>Unused</b>
    
year_built_messy = pd.DataFrame(year_built_array, columns=["year_built"])<br>
year_built_clean = year_built_messy["year_built"].str.replace('', np.nan)<br>
year_built_num = year_built_messy.astype(int)

year_built_num.head()
</div>

In [21]:
year_built_messy = pd.DataFrame(year_built_array, columns=["year_built"])
#year_built_messy = pd.to_numeric(year_built_messy)
year_built_messy.head()

,year_built
0,1970
1,2018
2,2016
3,2014
4,1959


In [22]:
year_built_clean = year_built_messy["year_built"].replace('', np.nan)

In [23]:
year_built_clean.head(10)

0    1970
1    2018
2    2016
3    2014
4    1959
5    1981
6    1989
7    1987
8    2001
9    2020
Name: year_built, dtype: object

In [24]:
year_built_digits = []

for i in year_built_clean:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        #year_built_clean.replace(i, np.nan)
        year_built_digits.append(np.nan)
    if ((str(i) == ' 9 2') or (str(i) == ' 8 2') or (str(i) == ' 2 2') or (str(i) == 'nasq') or (str(i) == ' 6 R') or (str(i) == ' 6 S')):
        year_built_digits.append(np.nan)
    else:
        year_built_digits.append(i)

In [25]:
year_built_digits[:10]

['1970',
 '2018',
 '2016',
 '2014',
 '1959',
 '1981',
 '1989',
 '1987',
 '2001',
 '2020']

In [26]:
year_built_num = pd.to_numeric(year_built_digits)
year_built_num[:10]

array([1970., 2018., 2016., 2014., 1959., 1981., 1989., 1987., 2001.,
       2020.])

In [27]:
year_built_num_pd = pd.DataFrame(year_built_num)

In [28]:
years_old = 2020 - year_built_num
years_old[:10]

array([50.,  2.,  4.,  6., 61., 39., 31., 33., 19.,  0.])

In [29]:
years_old_df = pd.DataFrame(years_old)

In [30]:
latitude_digits = []

for i in latitude_array_messy:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        latitude_digits.append(np.nan)
    else:
        latitude_digits.append(i)

In [31]:
longitude_digits = []

for i in longitude_array_messy:
    if ((str(i) >= 'A' and str(i) <= 'Z') or (str(i) >= 'a' and str(i) <= 'z')):
        longitude_digits.append(np.nan)
    else:
        longitude_digits.append(i)

In [32]:
latitude_digits = pd.to_numeric(latitude_digits)
longitude_digits = pd.to_numeric(longitude_digits)

In [33]:
payment_monthly_messy = pd.DataFrame(payment_monthly, columns=["monthly_payments"])
payment_monthly_messy.head()

,monthly_payments
0,$1469.34
1,$1941.05
2,$1102.22
3,$1513.01
4,$1681.22


In [34]:
payment_monthly_messy = payment_monthly_messy["monthly_payments"].str.replace('$', '')
payment_monthly_messy.head(10)

0    1469.34
1    1941.05
2    1102.22
3    1513.01
4    1681.22
5    2805.49
6    3022.56
7    1599.49
8    2378.27
9    2680.96
Name: monthly_payments, dtype: object

In [35]:
payment_monthly_messy = payment_monthly_messy.replace('', np.nan)

In [36]:
payment_monthly_clean = payment_monthly_messy.replace('None', np.nan)
payment_monthly_clean = pd.to_numeric(payment_monthly_clean)

<div class="alert alert-success">
    <b>Guide</b>
    
https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude

#### approximate radius of earth in km
R = 6373.0
    
lat1 = radians(52.2296756)<br>
lon1 = radians(21.0122287)

lat2 = radians(52.406374)<br>
lon2 = radians(16.9251681)

dlon = lon2 - lon1<br>
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2<br>
c = 2 * atan2(sqrt(a), sqrt(1 - a))

distance = R * c

print("Result:", distance)
    
#### Calgary Downtown coordinates (51.0449485,-114.0739559)
</div>

In [37]:
distance_downtown = []

for i, j in zip(latitude_digits, longitude_digits):
    lat1 = radians(i)
    lon1 = radians(j)
    lat2 = radians(51.0449485)
    lon2 = radians(-114.0739559)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    R = 6373.0
    distance = (R * c)
    
    distance_downtown.append(distance)

In [38]:
distance_downtown[:5]

[7.005098419705665,
 21.197854554893592,
 20.848932755553843,
 1.2391923716258482,
 6.3095902483495045]

In [39]:
distance_downtown_df = pd.DataFrame(distance_downtown)

<div class="alert alert-warning">
    <b>Another example</b>
    
import geopy.distance

coords_1 = (52.2296756, 21.0122287)<br>
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.vincenty(coords_1, coords_2).km)
</div>

In [40]:
more_details = pd.DataFrame([latitude_digits, longitude_digits, year_built_num]).T
#more_details.columns = ["postal_code", "latitude", "longitude", "year_built"]
more_details.head()

,0,1,2
0,51.024280,-113.979354,1970.0
1,50.868970,-113.957824,2018.0
2,50.869268,-113.970215,2016.0
3,51.053719,-114.084884,2014.0
4,51.030723,-113.986627,1959.0


In [41]:
lot_size_df = pd.DataFrame(lot_size)

In [42]:
basement_df = pd.DataFrame(basement)

In [43]:
garage_df = pd.DataFrame(garage)

In [44]:
more_df = pd.concat([postal_code_array_clean, more_details, years_old_df, distance_downtown_df, payment_monthly_clean, lot_size_df, basement_df, garage_df], axis=1)
more_df.columns = ["postal_code", "latitude", "longitude", "year_built", "years_old", "distance_downtown", "monthly_payments", "lot_size", "basement", "garage"]
more_df.head()

,postal_code,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments,lot_size,basement,garage
0,T2B1W3,51.024280,-113.979354,1970.0,50.0,7.005098,1469.34,4402,1.0,1.0
1,T3M2V8,50.868970,-113.957824,2018.0,2.0,21.197855,1941.05,2831,1.0,0.0
2,T3M2L7,50.869268,-113.970215,2016.0,4.0,20.848933,1102.22,NaN,0.0,1.0
3,T2N4H7,51.053719,-114.084884,2014.0,6.0,1.239192,1513.01,NaN,0.0,0.0
4,T2B1S8,51.030723,-113.986627,1959.0,61.0,6.309590,1681.22,5499,1.0,1.0


In [47]:
more_df.to_csv('data/calgary_listings-b.csv', index=False)

In [45]:
result_dup = pd.merge(calgary_df, more_df, on='postal_code')
result = result_dup.drop_duplicates()
result = result.reset_index(drop=True)
result.head(20)

,price,postal_code,address,bed,full_bath,half_bath,property_area,property_type,latitude,longitude,year_built,years_old,distance_downtown,monthly_payments,lot_size,basement,garage
0,339800,T2B1W3,86 DOVERCLIFFE CLOSE SE,4.0,2.0,NaN,1097.0,house,51.024280,-113.979354,1970.0,50.0,7.005098,1469.34,4402,1.0,1.0
1,339800,T2B1W3,86 DOVERCLIFFE CLOSE SE,4.0,2.0,NaN,1097.0,house,51.024280,-113.979354,1898.0,122.0,7.005098,NaN,NaN,0.0,0.0
2,339800,T2B1W3,86 DOVERCLIFFE CLOSE SE,4.0,2.0,NaN,1097.0,house,51.024280,-113.979354,1995.0,25.0,7.005098,NaN,NaN,0.0,0.0
3,339800,T2B1W3,86 DOVERCLIFFE CLOSE SE,4.0,2.0,NaN,1097.0,house,51.024280,-113.979354,1968.0,52.0,7.005098,NaN,NaN,0.0,0.0
4,339800,T2B1W3,86 DOVERCLIFFE CLOSE SE,4.0,2.0,NaN,1097.0,house,51.024280,-113.979354,2002.0,18.0,7.005098,NaN,NaN,0.0,0.0
5,339800,T2B1W3,86 DOVERCLIFFE CLOSE SE,4.0,2.0,NaN,1097.0,house,51.024280,-113.979354,NaN,NaN,7.005098,NaN,NaN,0.0,0.0
6,339800,T2B1W3,86 DOVERCLIFFE CLOSE SE,4.0,2.0,NaN,1097.0,house,51.024280,-113.979354,2014.0,6.0,7.005098,NaN,NaN,0.0,0.0
7,339800,T2B1W3,86 DOVERCLIFFE CLOSE SE,4.0,2.0,NaN,1097.0,house,51.024280,-113.979354,2000.0,20.0,7.005098,NaN,NaN,0.0,0.0
8,339800,T2B1W3,86 DOVERCLIFFE CLOSE SE,4.0,2.0,NaN,1097.0,house,51.024280,-113.979354,2017.0,3.0,7.005098,NaN,NaN,0.0,0.0
9,339800,T2B1W3,86 DOVERCLIFFE CLOSE SE,4.0,2.0,NaN,1097.0,house,51.024280,-113.979354,1974.0,46.0,7.005098,NaN,NaN,0.0,0.0


In [48]:
result.to_csv('data/calgary_listings.csv', index=False)

-----------

<div class="alert alert-info">
    <b>UNSED</b>
</div>

In [ ]:
house_details_more=[]
url_house = []
try:
    for detail in soup.find_all('a'):
        house_details_more.append(detail.get('href'))
except:
    house_details_more.append('None')
    
for i in house_details_more:
    if '/ab/calgary-real-estate/' in str(i):
        link = 'https://www.remax.ca' + str(i)
        url_house.append(link)
url_house

In [ ]:
url_house[:5]

In [ ]:
PostalCode = []
Latitude = []
Longitude = []
YearBuilt = []

for i in url_house[:5]:
    
    time.sleep(5)
    
    page = requests.get(i)
    soup = BeautifulSoup(page.text, "html.parser")
    
    lat_all = soup.find_all('script')[4]
    
    string = ', '.join(lat_all)
    pc = "postalCode"
    position_pc = string.find(pc)
    pcd = string[(position_pc+14):(position_pc+21)]
    PostalCode.append(pcd)

    lat = "latitude"
    position_lat = string.find(lat)
    latd = string[(position_lat+11):(position_lat+20)]
    Latitude.append(latd)

    long = "longitude"
    position_long = string.find(long)
    longd = string[(position_long+12):(position_long+23)]
    Longitude.append(longd)
    
    house_address = []
    
    time.sleep(5)
    
    try:
        addresses = soup.find_all('section', class_='property-details')
        for address in addresses:
            house_address.append(address.text)
    except:
        house_address.append('None')
    
    time.sleep(5)
    
    string = ', '.join(house_address)
    s_year = "Year Built"
    position_year = string.find(s_year)
    year_built = string[(position_year+12):(position_year+16)]
    YearBuilt.append(year_built)